In [1]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np

In [2]:
df = pd.read_csv("train.csv")

In [7]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
df.iloc[0]['comment_text']


"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [3]:
from tensorflow.keras.layers import TextVectorization


In [4]:
X = df['comment_text']
Y = df[df.columns[2:]].values


In [41]:
df.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [42]:
df[df.columns[2:]].values

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]], dtype=int64)

In [5]:
MAX_FEATURES = 200000


In [6]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                              output_sequence_length=1800,
                              output_mode='int')


In [7]:
vectorizer.adapt(X.values)


In [17]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [8]:
vectorized_text = vectorizer(X.values)


In [48]:
len(X)

159571

In [49]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [50]:
tf.data.Dataset.list_files

<function tensorflow.python.data.ops.dataset_ops.DatasetV2.list_files(file_pattern, shuffle=None, seed=None, name=None)>

In [9]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, Y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8)


In [10]:
batch_X, batch_Y = dataset.as_numpy_iterator().next()

In [33]:
batch_Y.shape

(16, 6)

In [54]:
int(len(dataset)*.7)

6981

In [10]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))


In [11]:
train_generator = train.as_numpy_iterator()


In [18]:
train_generator.next


<bound method _NumpyIterator.next of <tensorflow.python.data.ops.dataset_ops._NumpyIterator object at 0x000001BA70568DC0>>

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [13]:
model = Sequential()
model.add(Embedding(MAX_FEATURES*1, 32))
model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(6, activation='sigmoid'))

In [14]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400000   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [22]:
history = model.fit(train, epochs=12, validation_data=val)

Epoch 1/12
6981/6981 [==============================] - 4215s 603ms/step - loss: 0.0621 - val_loss: 0.0450
Epoch 2/12
6981/6981 [==============================] - 4107s 588ms/step - loss: 0.0458 - val_loss: 0.0411
Epoch 3/12
6981/6981 [==============================] - 4122s 590ms/step - loss: 0.0407 - val_loss: 0.0371
Epoch 4/12
6981/6981 [==============================] - 4119s 590ms/step - loss: 0.0365 - val_loss: 0.0332
Epoch 5/12
6981/6981 [==============================] - 4135s 592ms/step - loss: 0.0330 - val_loss: 0.0281
Epoch 6/12
6981/6981 [==============================] - 4152s 595ms/step - loss: 0.0299 - val_loss: 0.0266
Epoch 7/12
6981/6981 [==============================] - 4166s 597ms/step - loss: 0.0266 - val_loss: 0.0230
Epoch 8/12
6981/6981 [==============================] - 4174s 598ms/step - loss: 0.0241 - val_loss: 0.0222
Epoch 9/12
6981/6981 [==============================] - 4178s 599ms/step - loss: 0.0216 - val_loss: 0.0175
Epoch 10/12
6981/6981 [==============

In [15]:
from matplotlib import pyplot as plt


In [17]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()


NameError: name 'history' is not defined

<Figure size 800x500 with 0 Axes>

In [16]:
input_text = vectorizer('You freaking suck!')


In [17]:
batch = test.as_numpy_iterator().next()

In [18]:
batch_X, batch_Y = test.as_numpy_iterator().next()

In [21]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 1s 647ms/step


array([[1, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1],
       [1, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1],
       [1, 0, 0, 1, 0, 1],
       [1, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 0, 1],
       [1, 1, 0, 1, 0, 1],
       [0, 0, 0, 1, 0, 1],
       [1, 0, 0, 1, 1, 1],
       [1, 0, 0, 1, 0, 1],
       [1, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1]])

In [22]:
model.predict(batch_X)

1/1 [==============================] - 0s 90ms/step


array([[0.5001477 , 0.49958748, 0.49912855, 0.50231236, 0.49994716,
        0.5004272 ],
       [0.5001952 , 0.49854556, 0.49905553, 0.50327027, 0.49939582,
        0.5010137 ],
       [0.500142  , 0.49971303, 0.49912965, 0.501692  , 0.5006925 ,
        0.50068873],
       [0.5001817 , 0.4997553 , 0.4983431 , 0.5031032 , 0.49912828,
        0.50155944],
       [0.5003009 , 0.49999285, 0.4994479 , 0.50273645, 0.49972787,
        0.5007901 ],
       [0.49973074, 0.49947202, 0.49920705, 0.5029926 , 0.5000429 ,
        0.50122654],
       [0.5002529 , 0.5002978 , 0.4993161 , 0.5025974 , 0.49899754,
        0.5005584 ],
       [0.5002162 , 0.49980882, 0.4989711 , 0.50288856, 0.4988021 ,
        0.5008346 ],
       [0.50016975, 0.50049347, 0.4990039 , 0.5017077 , 0.500155  ,
        0.5005979 ],
       [0.50018984, 0.50026155, 0.4999038 , 0.50262785, 0.4995814 ,
        0.5008205 ],
       [0.5000294 , 0.50037575, 0.49892807, 0.5023752 , 0.49970537,
        0.5007933 ],
       [0.49992713, 0

In [19]:
res = model.predict(batch_X)


1/1 [==============================] - 1s 792ms/step


In [29]:
res.shape


(16, 6)

In [20]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [21]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [22]:
for batch in test.as_numpy_iterator():
    X_true, Y_true = batch
    yhat = model.predict(X_true)

    Y_true = Y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(Y_true, yhat)
    re.update_state(Y_true, yhat)
    acc.update_state(Y_true, yhat)

1/1 [==============================] - 0s 104ms/step


KeyboardInterrupt: 

In [23]:
print(f'Precision: {pre.result().numpy()},Recall: {re.result().numpy()},Accuracy:{acc.result().numpy()}')

Precision: 0.04135338217020035,Recall: 0.3492063581943512,Accuracy:0.0


In [38]:
pip install gradio

Note: you may need to restart the kernel to use updated packages.


In [24]:
import gradio as gr

In [36]:
model.save('toxicity.h5')

In [25]:
model = tf.keras.models.load_model('toxicity.h5')

In [26]:
input_str = vectorizer('hey i freanken hate you !')

In [27]:
res = model.predict(np.expand_dims(input_str, 0 ))

1/1 [==============================] - 1s 572ms/step


In [38]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [41]:
res

array([[0.9143855 , 0.00295345, 0.03172924, 0.00164384, 0.5728954 ,
        0.01907285]], dtype=float32)

In [28]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)

    return text

In [29]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.inputs.Textbox(lines=2, placeholder='comment to score'),
                         outputs='text')

C:\Python310\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
C:\Python310\lib\site-packages\gradio\inputs.py:30: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
C:\Python310\lib\site-packages\gradio\inputs.py:30: UserWarning: `numeric` parameter is deprecated, and it has no effect
  super().__init__(


In [30]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app


1/1 [==============================] - 0s 67ms/step
